<a href="https://colab.research.google.com/github/nurcann23/AygazCompVisionBootcamp/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Sizin gösterdiğiniz şekilde data kaynaktan indirildi.

In [4]:

import kagglehub
rrebirrth_animals_with_attributes_2_path = kagglehub.dataset_download('rrebirrth/animals-with-attributes-2')

print('Data source import complete.')

100%|██████████| 13.0G/13.0G [02:16<00:00, 102MB/s]

Extracting files...


Data source import complete.


- Yine sizin verdiğiniz kod ile dosyalardaki gerekli resimlerin path'leri dictionary'ye kaydedildi. Kod her hayvandan ilk 650 resmi alacak şekilde modifiye edildi.

In [5]:
import os

# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "/root/.cache/kagglehub/datasets/rrebirrth/animals-with-attributes-2/versions/1/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        # Check if the current directory contains the animal's name
        if animal in dirname:
            # Initialize the list if the animal is encountered for the first time
            if animal not in image_paths:
                image_paths[animal] = []
            # Add all image paths for the current animal
            i = 0;
            for filename in filenames:
                if i < 650:
                    image_paths[animal].append(os.path.join(dirname, filename))
                    i += 1
                else:
                    break


- Tüm resimler aynı boyuta getirildi ve normalized edildi. Burada normalde hakim olduğum OpenCV'yi kullanmıştım ama GPU kullanımını sağlayabilmek için tensorflow kullandım.
- Augmentation için daha önce Computer Vision dersimde yazmış olduğum add_salt_pepper_noise algoritmasını kullandım. Resimlere noise (resimlerin 15% kısmı noise olacak şekilde) ekleyerek datasetine ekledim.

In [6]:
import tensorflow as tf

def add_salt_pepper_noise(image):

    salt_pepper_image = image.copy()
    rows, cols, _ = salt_pepper_image.shape

    for x in range(rows):
        for y in range(cols):
            noisy_or_not = np.random.random()
            if noisy_or_not < 0.15:
                black_or_white = np.random.random()
                salt_pepper_image[x, y] = 0 if black_or_white < 0.5 else 255

    return salt_pepper_image

normalized_data = []

for animal, paths in image_paths.items():
    for img_path in paths:
        img_raw = tf.io.read_file(img_path)
        img_tensor = tf.image.decode_jpeg(img_raw, channels=3)
        img_resized = tf.image.resize(img_tensor, (128, 128))
        img_resized_np = img_resized.numpy()

        img_normalized = img_resized_np / 255.0
        normalized_data.append((img_normalized, animal))

        # GPU aşımından dolayı noise eklediğim halini deneyemedim.
        # Normalde amacım normal dataların yanına noise eklenmiş dataları eklemekti.

        #img_noisy = add_salt_pepper_noise(img_resized_np)
        #img_noisy_normalized = img_noisy / 255.0
        #normalized_data.append((img_noisy_normalized, animal))


- Task açıklamasında gösterildiği gibi resimler train ve test olarak ikiye ayrıldı.

In [7]:
from sklearn.model_selection import train_test_split

X = [x[0] for x in normalized_data]
y = [x[1] for x in normalized_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Eğitim setindeki resim sayısı: ", len(X_train))
print("Test setindeki resim sayısı: ", len(X_test))

Eğitim setindeki resim sayısı:  4550
Test setindeki resim sayısı:  1950


- Bu modelde ilk olarak 16 filtreli bir Conv2D katmanıyla başladım ve daha karmaşık özellikleri öğrenebilmek için 64 ve 128 filtreli Conv2D katmanlarını ekleyerek devam ettim. Overfitting’i önlemek için Dropout katmanı ekledim. Modeli son halina getirmeden denemeler yaparak en yüksek doğruluğu yakaladığım halini kullandım.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │           9,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,508,938 (24.83 MB)

 Trainable params: 6,508,938 (24.83 MB)

 Non-trainable params: 0 (0.00 B)

None


- Burada modelimi eğittim. Modelin performansı, validation data kullanılarak her epoch sonunda ölçüldü. Modelin accuracy değeri artarak güzel bir seviyeye ulaştı ama val_accuracy değeri artarak ilerlese de çok yükselmedi.

In [9]:
from tensorflow.keras.utils import to_categorical
import numpy as np

class_mapping = {cls: i for i, cls in enumerate(animals)}
y_train_enc = to_categorical([class_mapping[label] for label in y_train], num_classes=10)
y_test_enc = to_categorical([class_mapping[label] for label in y_test], num_classes=10)

model.fit(np.array(X_train), np.array(y_train_enc), epochs=10, validation_data=(np.array(X_test), np.array(y_test_enc)))


Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 121s 836ms/step - accuracy: 0.1907 - loss: 2.1947 - val_accuracy: 0.3923 - val_loss: 1.7152
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 150s 896ms/step - accuracy: 0.4056 - loss: 1.7072 - val_accuracy: 0.5323 - val_loss: 1.4270
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 143s 902ms/step - accuracy: 0.5218 - loss: 1.3869 - val_accuracy: 0.5949 - val_loss: 1.2005
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 131s 821ms/step - accuracy: 0.6096 - loss: 1.1431 - val_accuracy: 0.6133 - val_loss: 1.1150
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 152s 887ms/step - accuracy: 0.6653 - loss: 0.9571 - val_accuracy: 0.6205 - val_loss: 1.1704
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 134s 834ms/step - accuracy: 0.7307 - loss: 0.7776 - val_accuracy: 0.6318 - val_loss: 1.1482
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 142s 836ms/step - accuracy: 0.7877 - loss: 0.6293 - val_accuracy: 0.6492 - val_loss: 1.1350
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 142s 837ms/step - accuracy: 0.8302 -

- Bu kod parçasında adjust_brightness fonksiyonu kullanılarak her bir test resminin parlaklığıyla oynadım. Yeni bir test datası oluşturdum.

In [14]:
def get_manipulated_images(image, factor):
    return tf.image.adjust_brightness(image, factor)

manipulated_test_data = []
for img, label in zip(X_test, y_test):
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
    manipulated_img = get_manipulated_images(img_tensor, factor=0.2)
    manipulated_test_data.append((manipulated_img.numpy(), label))


- Manipüle edilen test verileri ile modeli test ettim ve doğruluk oranında yarı yarıya bir düşüş gözlemledim. Model bu manipülasyonla iyi sonuç vermedi.

In [16]:
manipulated_X_test = [x[0] for x in manipulated_test_data]
manipulated_y_test = [x[1] for x in manipulated_test_data]
manipulated_y_test_enc = to_categorical(
    [class_mapping[label] for label in manipulated_y_test], num_classes=10
)

loss, accuracy = model.evaluate(
    np.array(manipulated_X_test), np.array(manipulated_y_test_enc)
)


61/61 ━━━━━━━━━━━━━━━━━━━━ 12s 204ms/step - accuracy: 0.4779 - loss: 2.6188


- Bu kodda, manipüle edilmiş test verilerine renk sabitleme işlemi uyguladım. get_wb_images fonksiyonu, her bir görüntü kanalını kendi ortalamasına bölerek renk dengesini sağlar ve renk sapmalarını azaltır. Manipüle edilmiş her görüntü işlenip yeni bir listeye eklenmiştir.

In [18]:
def get_wb_images(image):

    mean_r = tf.reduce_mean(image[:, :, 0])
    mean_g = tf.reduce_mean(image[:, :, 1])
    mean_b = tf.reduce_mean(image[:, :, 2])
    balanced_image = tf.stack([
        image[:, :, 0] / mean_r,
        image[:, :, 1] / mean_g,
        image[:, :, 2] / mean_b
    ], axis=2)

    balanced_image = tf.clip_by_value(balanced_image * 128, 0, 255)
    return balanced_image

wb_test_data = []
for img, label in manipulated_test_data:
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
    wb_img = get_wb_images(img_tensor)
    wb_test_data.append((wb_img.numpy(), label))


- Model bu manipülasyonla daha da kötü bir sonuç verdi ve önceki işleme göre accuracy yine yarı yarıya düştü.

In [19]:
wb_X_test = [x[0] for x in wb_test_data]
wb_y_test = [x[1] for x in wb_test_data]
wb_y_test_enc = to_categorical(
    [class_mapping[label] for label in wb_y_test], num_classes=10
)

loss, accuracy = model.evaluate(
    np.array(wb_X_test), np.array(wb_y_test_enc)
)
print("Renk Sabitliği Uygulanmış Test Verisi - Loss:", loss)
print("Renk Sabitliği Uygulanmış Test Verisi - Accuracy:", accuracy)


61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 202ms/step - accuracy: 0.2570 - loss: 836.7116
Renk Sabitliği Uygulanmış Test Verisi - Loss: 792.370361328125
Renk Sabitliği Uygulanmış Test Verisi - Accuracy: 0.26615384221076965


- Bu sonuçlar, modelin manipülasyonlar sonrası doğruluğunun düştüğünü muhtemel olarak overfitting olduğunu gösteriyor.